# Chapter 4 Interpretable Models

일부 모델의 경우에는 그 자체로 해석이 쉬운 것들이 있다. 
- Linear Regression
- Logistic Regression
- Decision Tree

용어 설명
- 선형성은 feature와 target의 값이 선형적인 관계인 것
- 단조성은 feature와 target의 값이 한 방향으로만 일관되게 진행되는 것
- interaction은 복수 개의 feature사이의 관계가 모델링되는 것 (ex. (방 개수, 집 크기) => 집값 예측)

![](./images/ch04/01.png)



## Linear Regression
---

### 정의 및 구성
- target의 값이 feature 값의 가중합으로 예측되는 것
- weight for each feature, bias, error term following Gaussian dist
- weight esitmation coms from LSME
- weight estimation come with confidence interval 
  - 95% interval = 95 out of 100 try, the confidience interval include true weight
![](./images/ch04/02.png)


### 선형 모델이 가정하는 데이터 특징
- Linearity 
  - prediction as a linear combination of features
  - each term is additive (easy of seperation of the effect)
- Normality
  - target follows a normal distribution
  - If not, use Generalized Linear Model (GLM)
- Homoscedasticity (constant variable)
  - variance of error be constant over entire feature space
  - ex) 방크기 feature로 집값 예측시, 작은 방크기에서의 예측 오류와 큰 방 크기에서의 예측 오류가 다르면 안됨
- Independece
  - each instance is indep
  - ex) 동일한 환자에서 연속 피뽑은 샘플 => not independent
  - If not, use GEE
- Fiexed features
  - feature is not random variable, 
  - it is just constant, free of measurement errors
  - but unrealistic assumption
- Absence of multicollinearity 
  - It not, use interaction term


### 선형 모델의 해석
- For numeric feature
  - feature의 값을 1 unit 변화시키면, target 값이 weight만큼 변한다. 
- Binary feature
  - reference category로부터 해당 category로 변화시키면 weight만큼 변한다. 
  - ex) 날씨 = {Sunny, Cloudy}일 때 sunny가 reference
- Categorical feature
  - L-1 one-hot encoding
  - same as binary feature
- Intercept
  - meaningless case : all numeric features set to zero, all categorical feature to references
  - meaningful case : when all numeric featuees are standadised(mean of zero, deviation of one)
- R-squared
  - how much of the target variance is explained by the model
  - higher R-squared, better model explains the data
![](./images/ch04/03.png)

- Feature Importance
  - absolute value of t-statistic (estimated weight scaled with std)
  - the more variance of estimated weight => we are not sure of correct importance => make less feature importance

![](./images/ch04/04.png)

### Example ( prediction of # of rented bikes )
![](./images/ch04/05.png)

- ex) 1도 올라가면 110개의 자전거가 추가로 랜트된다. 
- ex) 좋은 날씨 대비 비가 오면 랜트수가 1901개 줄어든다. 
- 모든 해석은 다른 feature가 고정되었을 때를 가정한다. 
- 하나의 feature unit만 증가시키는 것을 때로는 unrealistic inout이 되고 만다. 
  - ex) increase of # of room without increasing size of room => 현실적으로 비존재할 수도
  
### Visual Interpretation

#### Weight Plot
- median값, interval의 zero 포함, interval의 길이 등을 고려
- 각 feature마다 scale이 다르다는 점이 문제이다. 
  - 정확히 비교할려면 모든 feature의 scale을 맞추어야 한다. 

![](./images/ch04/06.png)

#### Effect Plot
- effect = weight * feature value
- weight은 feature scale에 따라 차이가 나지만, effect는 scale-independent하다. 
- 카테고리 feature는 하나의 boxplot으로 summarized되서 표시

![](./images/ch04/07.png)


### Explain Individual Predictions

- 예를 들어서 다음의 feature를 갖는 하나의 data point를 설명해보자 
![](./images/ch04/08.png)

- red cross가 각 feature별 예측 기여도를 나타냄 
- 대조적인 설명
  - 평균치보다 낮은 주요 원인 - 낮은 온도, 2011년의 초반  
![](./images/ch04/09.png)

### Do Linear Models Create Good Explanations?

No because
- Contrastive한 설명을 할 수는 있지만, reference instance라는 것이 artificial, meaningless하다. 
  - 다만 모든 numerical feature가 standarized되고 모든 category feature가 effect coding된다면 reference instance는 일종의 평균 instance가 된다. 
  - 여전히 현실적이지 않을 수는 있지만, 상대적으로 more meaningful하다.
- selective한 설명이지 않다. 
  - 모든 feature를 다 동원해서 설명하므로, 
  - 대안으로 sparse linear model


## Sparse Linear Model
---
seletive한 설명을 위해 sparsity를 linear model 안으로 도입 

### LASSO (least absolute shrinkage selection operator)

최적화시에 큰 weight를 억제시키는 L1-norm 추가 
![](./images/ch04/10.png)

lambda가 크면 많은 feature weight이 0으로 된다.
- CV를 통해서 적절한 lambda를 정하자
![](./images/ch04/11.png)

### Example with Lasso

강한 lambda를 사용해서 2개의 feature만 살아남게 학습한 결과 
![](./images/ch04/12.png)

### Other Methods for Sparsity in Linear Models

Preprocessing을 통한 방법들
- manually selection of features by expert knowledge
- univariate selection like high correlation coefficient

Step-wise method
- forward selection : 소수의 feature로부터 시작해서 하나씩 feature를 추가해가면서 
- backward selection : 모든 feature를 다 쓴 것부터 시작해서 하나씩 빼가면서




## Logistic Regression

### What is Wrong with Linear Regression for Classification?

- linear model은 output probability를 출력해주지 못한다.
- predicted class를 일종의 숫자로 취급하여 이들을 구분할 hyperplane을 학습하는 것 
  - multiple calss로 확장이 안된다. label = 1,2,3 
- linear model은 0보다 작은 값, 1보다 큰 값도 외삽한다. 
![](./images/ch04/13.png)

### Theory

* logistic regression model uses the logistic function to squeeze the output of a linear equation between 0 and 1.
![](./images/ch04/14.png)

### Interpretation

Linear model for the log odds
![](./images/ch04/15.png)
![](./images/ch04/16.png)

when we increase one of the feature values by 1
![](./images/ch04/17.png)
![](./images/ch04/18.png)

